<a href="https://colab.research.google.com/github/BlackCranium/NNAQ_BC/blob/master/NNAQ_BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>источник: https://tproger.ru/articles/kejs-habib-vs-makgregor-ili-raspoznavanie-lic-na-python/?utm_medium=messenger&utm_source=telegram  
>Кейс: Хабиб vs МакГрегор или Распознавание лиц на Python


# Эксперимент (jupyther + colab + github + google drive)

подключим необходимые библиотеки. 
В Colab большинство библиотек уже установлено, поэтому нам осталось их только импортировать:


*   cv2 — для загрузки и обработки изображений;
*   dlib — для детекции лиц;
*   numpy — для работы с матрицами.


In [ ]:
import cv2, dlib
import numpy as np


In [5]:
def add(a,b):
  return a+b

add(2,6)

8